In [ ]:
!pip install -qU diffusers accelerate transformers huggingface_hub safetensors

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Model files and layouts

Diffusers stores model weights as safetensors files in **Diffusers-multifolder** layout.

## Files

### safetensors

`safetensors` is a safe and fast file format for securely storing and loading tensors. `safetensors` restricts the header size to limit certain types of attacks, support lazy loading (useful for distributed setups), and has generally faster loading speeds.

`safetensors` files can be organized as
1. Diffusers-multifolder layout: there may be several separate safetenosrs files, one for each pipeline component (text encoder, UNet, VAE), organized in subfolders.
2. single-file layout: all the model weights may be saved in a single file.

##### multifolder

load a model with safetensors files stored in multiple folders

In [ ]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained(
    'stable-diffusion-v1-5/stable-diffusion-v1-5',
    use_safetensors=True,
)

##### single file

load a model with all the weights stored in a single safetensors file.

In [ ]:
from diffusers import StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_single_file(
    "https://huggingface.co/WarriorMama777/OrangeMixs/blob/main/Models/AbyssOrangeMix/AbyssOrangeMix.safetensors"
)

#### LoRA files

LoRA is a lightweight adapter that is fast and easy to train, making them especially popular for generating images in a certain way or style. These adapters are commonly stored in a safetensors file, and are widely popular on model sharing platforms.

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

# base model
pipeline = StableDiffusionXLPipeline.from_pretrained(
    'Lykon/dreamshaper-xl-1-0',
    torch_dtype=torch.float16,
    variant='fp16',
).to('cuda')

In [ ]:
# download LoRA weights
!wget https://civitai.com/api.download/models/168776 -O blueprintify.safetensors

In [ ]:
# load LoRA
pipeline.load_lora_weights(
    '.',
    weight_name='blueprintify.safetensors'
)

In [ ]:
prompt = "bl3uprint, a highly detailed blueprint of the empire state building, explaining how to build all parts, many txt, blueprint grid backdrop"
negative_prompt = "lowres, cropped, worst quality, low quality, normal quality, artifacts, signature, watermark, username, blurry, more than one bridge, bad architecture"

image = pipeline(
    prompt=prompt,
    negative_prompt=negative_prompt,
    generator=torch.manual_seed(111),
).images[0]

image

### ckpt

PyTorch's `torch.save` function uses Python's `pickle` utility to serialize and save models. These files are saved as a `ckpt` file and contain the entire model's weights.

In [ ]:
from diffusers import StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_single_file(
    "https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/blob/main/v1-5-pruned.ckpt"
)

## Storage layout

### Diffusers-multifolder

The Diffusers-multifolder layout is the default storage layout for Diffusers. Each component’s (text encoder, UNet, VAE) weights are stored in a separate subfolder. The weights can be stored as safetensors or ckpt files.

Load from Diffusers-multifolder layout:

In [ ]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
).to("cuda")

Benefits of the Diffusers-multifolder layout:
1. Faster to load each component file individually or in parallel.
2. Reduced memory usage because we only load the components we need. For example, models like `SDXL Turbo`, `SDXL Lightning`, and `Hyper-SD` have the same components except for the UNet. We can reuse their shared components with the `from_pipe()` method and only load the UNet.

In [ ]:
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler

# download one model
sdxl_pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
).to('cuda')

In [ ]:
# switch UNet for another model
unet = UNet2DConditionModel.from_pretrained(
    'stabilityai/sdxl-turbo',
    subfolder='unet',
    torch_dtype=torch.float16,
    variant='fp16',
    use_safetensors=True,
)

In [ ]:
# reuse all the same components in new model except for the UNet
turbo_pipeline = StableDiffusionXLPipeline.from_pipe(
    sdxl_pipeline,
    unet=unet,
).to('cuda')
turbo_pipeline.scheduler = EulerDiscreteScheduler.from_config(
    turbo_pipeline.scheduler.config,
    timestep_spacing='trailing',
)

In [ ]:
image = turbo_pipeline(
    'an astronuat riding a unicorn on mars',
    num_inference_steps=1,
    guidance_scale=0.,
).images[0]

image

3. Reduced storage requirements because if a component, such as the SDXL VAE, is shared across multiple models, we only need to download and store a single copy of it instead of downloading and shtoring it multiple times.

4. Flexibility to replace a component in the model with a newer or better version.

In [ ]:
from diffusers import DiffusionPipeline, AutoencoderKL

vae = AutoencoderKL.from_pretrained(
    'madebyollin/sdxl-vae-fp16-fix',
    torch_dtype=torch.float16,
    use_safetensors=True,
)

pipeline = DiffusionPipeline.from_pretrained(
    'stabilityai/stable-diffusion-xl-base-1.0',
    vae=vae, # replace a newer vae
    torch_dtype=torch.float16,
    variant='fp16',
    use_safetensors=True,
).to('cuda')

5. More visibility and information about a model's components, which are stored in a `config.json` file in each component subfolder.

### Single-file

All the model component (text encoder, UNet, VAE) weights are kept together instead of separately in subfolders. This can be a safetensors or ckpt file.

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

pipeline = StableDiffusionXLPipeline.from_single_file(
    "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0.safetensors",
    torch_dtype=torch.float16,
    variant='fp16',
    use_safetensors=True,
).to('cuda')

Benefits of using a single-file layout:

1. Easy compatibility with diffusion interfaces sucha as `ComfyUI`
2. Easier to manage a single file.

## Convert layout and files

Check out [diffusers/scripts](https://github.com/huggingface/diffusers/tree/main/scripts) to find a script to convert to diffusers ecosystem.



## Single-file layout usage

### Load a pipeline or model

In [ ]:
# load a pipeline
from diffusers import StableDiffusionXLPipeline

ckpt_path = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0_0.9vae.safetensors"
pipeline = StableDiffusionXLPipeline.from_single_file(ckpt_path)

In [ ]:
# load a model
from diffusers import StableCascadeUNet

ckpt_path = "https://huggingface.co/stabilityai/stable-cascade/blob/main/stage_b_lite.safetensors"
model = StableCascadeUNet.from_single_file(ckpt_path)

Customize components in the pipeline by passing them directly to the `from_single_file()` method:

In [ ]:
from diffusers import StableDiffusionXLPipeline, DDIMScheduler

ckpt_path = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0_0.9vae.safetensors"
scheduler = DDIMScheduler()

pipline = StableDiffusionXLPipeline.from_single_file(
    ckpt_path,
    scheduler=scheduler,
)

Or we can use a ControlNet model in the pipeline

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel

ckpt_path = "https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/blob/main/v1-5-pruned-emaonly.safetensors"
controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_canny")

pipeline = StableDiffusionControlNetPipeline.from_single_file(
    ckpt_path,
    controlnet=controlnet,
)

### Customize configuration options

Models have a configuration file that define their attributes like the number of inputs in a UNet. Pipelines configuration options are available in the pipeline's class.

##### Hub configuration file

The `from_single_file()` automatically maps the checkpoint to the appropriate model repository, but there are cases where it is useful to use the `config` parameters.

We can also explicitly specify the configuration to use:

In [ ]:
from diffusers import StableDiffusionXLPipeline

ckpt_path = "https://huggingface.co/segmind/SSD-1B/blob/main/SSD-1B.safetensors"
repo_id = 'segmind/SSD-1B'

pipeline = StableDiffusionXLPipeline.from_single_file(
    ckpt_path,
    config=repo_id,
)

The model loads the configuration file for the UNet, VAE, and text encoder from their respective subfolders in the repository.

##### original configuration file

In [ ]:
from diffusers import StableDiffusionXLPipeline

ckpt_path = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0_0.9vae.safetensors"
original_config = "https://raw.githubusercontent.com/Stability-AI/generative-models/main/configs/inference/sd_xl_base.yaml"

pipeline = StableDiffusionXLPipeline.from_single_file(
    ckpt_path,
    original_config=original_config,
)

We can override the default parameters by providing the parameters directly to the `from_single_file()` method.

In [ ]:
# pipeline
from diffusers import StableDiffusionXLInstructPix2PixPipeline

ckpt_path = "https://huggingface.co/stabilityai/cosxl/blob/main/cosxl_edit.safetensors"

pipeline = StableDiffusionXLInstructPix2PixPipeline.from_single_file(
    ckpt_path,
    config='diffusers/sdxl-instructpix2pix-768',
    is_cosxl_edit=True,
)

In [ ]:
# model
from diffusers import UNet2DConditionModel

ckpt_path = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0_0.9vae.safetensors"

model = UNet2DConditionModel.from_single_file(
    ckpt_path,
    upcast_attention=True,
)

### Local files

The `from_single_file()` method also could configure a pipeline or mdoel by inferring the model type from the keys in the checkpoint file. The inferred model type is used to determine the appropriate model repository on the HuggingFace Hub to configure the model or pipeline.

For example, any single file checkpoint based on the Stable Diffusion XL base model will use the `stabilityai/stable-diffusion-xl-base-1.0` model repository to configure the pipeline.

We can also download the configuration files with the `snapshot_download` function and the model checkpoint with the `hf_hub_download` function if there is restricted internet access.

In [ ]:
# Hub cache directory
from huggingface_hub import snapshot_download, hf_hub_download

my_local_checkpoint_path = hf_hub_download(
    repo_id='segmind/SSD-1B',
    filename='SSD-1B.safetensors',
)

my_local_config_path = snapshot_download(
    repo_id='segmind/SSD-1B',
    allow_patterns=['*.json', '**/*.json', '*.txt', '**/*.txt'],
)

pipeline = StableDiffusionXLPipeline.from_single_file(
    my_local_checkpoint_path,
    config=my_local_config_path,
    local_files_only=True,
)

In [ ]:
# specific local directory
from huggingface_hub import snapshot_download, hf_hub_download

my_local_checkpoint_path = hf_hub_download(
    repo_id='segmind/SSD-1B',
    filename='SSD-1B.safetensors',
    local_dir='my_local_checkpoints',
)

my_local_config_path = snapshot_download(
    repo_id='segmind/SSD-1B',
    allow_patterns=['*.json', '**/*.json', '*.txt', '**/*.txt'],
    local_dir='my_local_configs',
)

pipeline = StableDiffusionXLPipeline.from_single_file(
    my_local_checkpoint_path,
    config=my_local_config_path,
    local_files_only=True,
)